In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
data = pd.read_csv("../input/vehicle-dataset-from-cardekho/Car details v3.csv")
data

In [ ]:
data.isnull().sum()

In [ ]:
data.name.value_counts()

In [ ]:
car=data.name.str.split(expand=True)
cartype=car[0]
data['cartype']=cartype
data

In [ ]:
data.cartype.value_counts()

In [ ]:
data = data.drop(['name'],axis=1)

In [ ]:
today=datetime.today()
curr_year=today.year
data['No. of years']=curr_year-data['year']
data=data.drop(['year'],axis=1)
data

In [ ]:
data.fuel.value_counts()

In [ ]:
data.seller_type.value_counts()

In [ ]:
data.transmission.value_counts()

In [ ]:
data.owner.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data.fuel=le.fit_transform(data.fuel)
data.seller_type=le.fit_transform(data.seller_type)
data.transmission=le.fit_transform(data.transmission)
data

In [ ]:
data=data.replace({'owner':{'First Owner':1,
                     'Second Owner':2,
                     'Third Owner':3,
                     'Fourth & Above Owner':4,
                      'Test Drive Car':5}})
data

In [ ]:
mileage_alter=data.mileage.str.split(expand=True)
mileage_new=mileage_alter[0]
data['Mileage']=mileage_new
data=data.drop(['mileage'],axis=1)
data

In [ ]:
engine_alter=data.engine.str.split(expand=True)
engine_new = engine_alter[0]
data['Engine']=engine_new
data=data.drop(['engine'],axis=1)
data

In [ ]:
power=data.max_power.str.split(expand=True)
data['Max_power']=power[0]
data=data.drop(['max_power'],axis=1)
data

In [ ]:
data.isnull().sum()

In [ ]:
data.seats.value_counts()

In [ ]:
data=data.drop(['seats'],axis=1)
data

In [ ]:
tor=data.torque.str.split('@',expand=True)
data['Torq']=tor[0]
data

In [ ]:
tor=data.Torq.str.split(expand=True)
data['Tor']=tor[0]
data

In [ ]:
tor=data.Torq.str.split(expand=True)
data['Tor']=tor[0]
data

In [ ]:
tor=data.Tor.str.split('N',expand=True)
data['torq']=tor[0]

data

In [ ]:
torq=data.torq.str.split('n',expand=True)
data['Torque']=torq[0]
data=data.drop(['torque','Torq','Tor','torq'],axis=1)
data

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(axis=0,inplace=True)
data

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr = data.corr()
plt.figure(figsize=(12,6))
sns.heatmap(corr,annot=True,cmap='YlGn')

In [ ]:
data=data.drop('cartype',axis=1)

In [ ]:
Torque=data.Torque.str.split('k',expand=True)
data['torque']=Torque[0]
data=data.drop(['Torque'],axis=1)
data

In [ ]:
torque=data.torque.str.split('(',expand=True)
data['Torque']=torque[0]
data=data.drop(['torque'],axis=1)
data

In [ ]:
data=data.apply(pd.to_numeric)
data

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr = data.corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr,annot=True,cmap='YlGn')

In [ ]:
sns.pairplot(corr)

In [ ]:
X=data.drop(['selling_price'],axis=1)
y=data.selling_price

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
etr=ExtraTreesRegressor()
etr.fit(X,y)

In [ ]:
etr.feature_importances_

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x=X.columns,y=etr.feature_importances_)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train.shape,X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train,y_train)

In [ ]:
import numpy as np

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(RandomForestRegressor(),{
    'n_estimators':[int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)],
    'max_features':['auto','sqrt'],
    'max_depth':[int(x) for x in np.linspace(5,30, num = 6)],
    'min_samples_split':[2,5,10,15,100],
    'min_samples_leaf':[1,2,5,10]}
     )

In [ ]:
rs.fit(X_train,y_train)

In [ ]:
rs.best_params_

In [ ]:
param = pd.DataFrame(rs.cv_results_)
param

In [ ]:
param[['param_n_estimators','param_min_samples_split','param_min_samples_leaf','param_max_features','param_max_depth','mean_test_score']]

In [ ]:
model = RandomForestRegressor(n_estimators=1000,
 min_samples_split= 2,
 min_samples_leaf= 2,
 max_features= 'auto',
 max_depth= 15)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
pred=model.predict(X_test)
metrics.r2_score(pred,y_test)

In [ ]:
plt.scatter(y_test,pred)